## Import Required Libraries

In [19]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import time

## Define Sector ETFs and Their Details

In [20]:
# Define sector ETFs with their sector names
sector_etfs = {
    'XLK': 'Technology',
    'XLV': 'Healthcare',
    'XLF': 'Financial',
    'XLI': 'Industrial',
    'XLY': 'Consumer Discretionary'
}

## Function to Get ETF Holdings from Yahoo Finance
This function retrieves the current holdings of a given ETF using yfinance - completely free and no API key needed!

In [14]:
def get_etf_holdings(etf_ticker):
    """
    Get holdings for a given ETF ticker using yfinance.
    
    Parameters:
    etf_ticker (str): The ticker symbol of the ETF
    
    Returns:
    list: List of tuples (symbol, weight) of holdings in the ETF
    """
    try:
        etf = yf.Ticker(etf_ticker)
        
        # Get the holdings - yfinance provides this for many ETFs
        holdings = etf.get_holdings()
        
        if holdings is not None and not holdings.empty:
            # Extract symbols from the holdings dataframe
            # The dataframe typically has 'Symbol' or similar column
            if 'Symbol' in holdings.columns:
                symbols = holdings['Symbol'].tolist()
            elif 'symbol' in holdings.columns:
                symbols = holdings['symbol'].tolist()
            else:
                # If Symbol column not found, try the index
                symbols = holdings.index.tolist()
            
            print(f"Found {len(symbols)} holdings for {etf_ticker}")
            return symbols
        else:
            print(f"No holdings data available for {etf_ticker} via yfinance")
            return []
        
    except Exception as e:
        print(f"Error fetching holdings for {etf_ticker}: {str(e)}")
        return []

## Automated Update Function Using Yahoo Finance
This function fetches and updates holdings for a specific sector ETF.

In [ ]:
def update_sector_from_yahoo(etf_code):
    """
    Automatically fetch and update holdings for a sector ETF from Yahoo Finance.
    
    Parameters:
    etf_code (str): The ETF ticker (e.g., 'XLK')
    
    Returns:
    pd.DataFrame: DataFrame with the sector holdings
    """
    if etf_code not in sector_etfs:
        print(f"Unknown ETF code: {etf_code}")
        return None
    
    sector_name = sector_etfs[etf_code]
    print(f"\nFetching holdings for {etf_code} ({sector_name})...")
    
    symbols = get_etf_holdings(etf_code)
    
    if symbols:
        df = pd.DataFrame({
            'Symbol': symbols,
            'Sector': sector_name,
            'Sector_Code': etf_code
        })
        print(f"Successfully created dataframe with {len(df)} stocks")
        return df
    else:
        print(f"Failed to fetch holdings for {etf_code}")
        return None

## Update All Sectors at Once
Fetch all ETF holdings from Yahoo Finance and update the entire CSV file.

In [16]:
def update_all_sectors_from_yahoo():
    """
    Fetch holdings for all sector ETFs from Yahoo Finance and create a complete dataframe.
    
    Returns:
    pd.DataFrame: Complete dataframe with all sectors, or None if all fetches fail
    """
    all_holdings = []
    
    for etf_code in sector_etfs.keys():
        holdings_df = update_sector_from_yahoo(etf_code)
        if holdings_df is not None:
            all_holdings.append(holdings_df)
        time.sleep(1)  # Be polite to Yahoo Finance servers
    
    if all_holdings:
        complete_df = pd.concat(all_holdings, ignore_index=True)
        complete_df = complete_df.sort_values(['Sector_Code', 'Symbol']).reset_index(drop=True)
        
        print(f"\n{'='*60}")
        print(f"Successfully fetched {len(complete_df)} total stocks")
        print(f"\nBreakdown by sector:")
        print(complete_df.groupby(['Sector_Code', 'Sector']).size())
        print(f"{'='*60}")
        
        return complete_df
    else:
        print("Failed to fetch any holdings data")
        return None

## Alternative: Manual Input Function
Since ETF holdings data might not be  available this function allows manual updates.

In [4]:
def update_sector_holdings_manual(etf_code, sector_name, symbols_list):
    """
    Manually update holdings for a specific sector.
    
    Parameters:
    etf_code (str): The ETF ticker (e.g., 'XLK')
    sector_name (str): The sector name (e.g., 'Technology')
    symbols_list (list): List of stock symbols
    
    Returns:
    pd.DataFrame: DataFrame with the sector holdings
    """
    df = pd.DataFrame({
        'Symbol': symbols_list,
        'Sector': sector_name,
        'Sector_Code': etf_code
    })
    return df

## Load Current CSV File

In [17]:
# Load the current sector stocks CSV
csv_path = 'sector_stocks.csv'

try:
    current_df = pd.read_csv(csv_path)
    print(f"Current CSV loaded successfully with {len(current_df)} stocks")
    print(f"\nStocks per sector:")
    print(current_df.groupby('Sector_Code').size())
    print(f"\nFirst few rows:")
    display(current_df.head())
except FileNotFoundError:
    print(f"CSV file not found at {csv_path}")
    current_df = pd.DataFrame(columns=['Symbol', 'Sector', 'Sector_Code'])

Current CSV loaded successfully with 307 stocks

Stocks per sector:
Sector_Code
XLF    66
XLI    66
XLK    62
XLV    64
XLY    49
dtype: int64

First few rows:


,Symbol,Sector,Sector_Code
0,AAPL,Technology,XLK
1,MSFT,Technology,XLK
2,NVDA,Technology,XLK
3,AVGO,Technology,XLK
4,CRM,Technology,XLK


## Manual Update Example
Use this cell to manually add or update stocks for a specific sector.

In [ ]:
# Example: Manually update Technology sector (XLK)

# new_xlk_symbols = ['AAPL', 'MSFT', 'NVDA', 'AVGO', 'CRM', 'ORCL', 'CSCO']
# xlk_update = update_sector_holdings_manual('XLK', 'Technology', new_xlk_symbols)
# display(xlk_update)

## Update Specific Sector
Function to update holdings for a specific sector ETF.

In [ ]:
def update_sector_in_csv(df, new_holdings_df, sector_code):
    """
    Update the CSV dataframe with new holdings for a specific sector.
    
    Parameters:
    df (pd.DataFrame): The current dataframe
    new_holdings_df (pd.DataFrame): New holdings dataframe
    sector_code (str): The sector code to update (e.g., 'XLK')
    
    Returns:
    pd.DataFrame: Updated dataframe
    """
    # Remove old entries for this sector
    df_filtered = df[df['Sector_Code'] != sector_code]
    
    # Add new holdings
    df_updated = pd.concat([df_filtered, new_holdings_df], ignore_index=True)
    
    # Sort by sector code and symbol
    df_updated = df_updated.sort_values(['Sector_Code', 'Symbol']).reset_index(drop=True)
    
    print(f"Updated {sector_code}: {len(new_holdings_df)} stocks")
    
    return df_updated

## Save Updated CSV
Save the updated dataframe back to the CSV file.

In [ ]:
def save_updated_csv(df, filepath='sector_stocks.csv', backup=True):
    """
    Save the updated dataframe to CSV.
    
    Parameters:
    df (pd.DataFrame): The dataframe to save
    filepath (str): Path to save the CSV
    backup (bool): Whether to create a backup of the old file
    """
    if backup:
        # Create backup with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        backup_path = f'sector_stocks_backup_{timestamp}.csv'
        try:
            current_df = pd.read_csv(filepath)
            current_df.to_csv(backup_path, index=False)
            print(f"Backup created: {backup_path}")
        except:
            print("No existing file to backup")
    
    # Save updated file
    df.to_csv(filepath, index=False)
    print(f"Updated CSV saved to {filepath}")
    print(f"Total stocks: {len(df)}")
    print(f"\nBreakdown by sector:")
    print(df.groupby(['Sector_Code', 'Sector']).size())

## Complete Update Workflow Example
Example workflow to update all sectors.

In [ ]:
# Example: Update a specific sector

# Step 1: Get new holdings (using your preferred method)
# new_symbols = ['AAPL', 'MSFT', 'NVDA']  # Replace with actual holdings
# new_holdings = update_sector_holdings_manual('XLK', 'Technology', new_symbols)

# Step 2: Update the dataframe
# updated_df = update_sector_in_csv(current_df, new_holdings, 'XLK')

# Step 3: Save to CSV
# save_updated_csv(updated_df)

## Verification
Verify the updated CSV file.

In [18]:
# Verify the updated CSV
verify_df = pd.read_csv('sector_stocks.csv')

print(f"Total stocks in updated file: {len(verify_df)}")
print(f"\nStocks by sector:")
print(verify_df.groupby(['Sector_Code', 'Sector']).size())
print(f"\nSample data:")
display(verify_df.groupby('Sector_Code').head(3))

Total stocks in updated file: 307

Stocks by sector:
Sector_Code  Sector                
XLF          Financial                 66
XLI          Industrial                66
XLK          Technology                62
XLV          Healthcare                64
XLY          Consumer Discretionary    49
dtype: int64

Sample data:


,Symbol,Sector,Sector_Code
0,AAPL,Technology,XLK
1,MSFT,Technology,XLK
2,NVDA,Technology,XLK
62,UNH,Healthcare,XLV
63,JNJ,Healthcare,XLV
64,LLY,Healthcare,XLV
126,JPM,Financial,XLF
127,BAC,Financial,XLF
128,WFC,Financial,XLF
192,GE,Industrial,XLI
